In [66]:
import pandas as pd
import numpy as np
import re

In [2]:
df = pd.read_excel('spamdata.xlsx')

In [7]:
df['MESSAGE'].head()

0    Happy New Year!&nbsp;<br />\r\n新年快樂<br />\r\n<...
1    您好, 我們是參加2017HKFW的台灣廠商京澔公司(攤位號3D-DS140)。目前正在尋找...
2    Please quote me the latest price of the above ...
3    A buyer with the below contact info has just r...
4    Please quote me the latest price of the above ...
Name: MESSAGE, dtype: object

In [43]:
df['MESSAGE'].iloc[5]

'Please quote me the latest price of the above product.<br><br>Purchase Quantity: 3000 piece(s)<br><br><br />\r\nSir,<br />\r\n<br />\r\n&nbsp;<br />\r\n<br />\r\n<br />\r\n<br />\r\nWe are interested to purchase your supplyment product.<br />\r\n<br />\r\n<br />\r\n<br />\r\nKindly send your quotations and price list to us.<br />\r\n<br />\r\n<br />\r\n<br />\r\nFor more details about this order, please contact:Mr Desmond Attah.<br />\r\n<br />\r\nEmail:rita_izus@lovecat.com<br />\r\nEmail:rita_salicheconsultant@consultant.com<br />\r\n<br />\r\nWaiting to hear from you asap.<br />\r\n<br />\r\n&nbsp;<br />\r\n<br />\r\nThanks<br />\r\n<br />\r\n<br />\r\nMs Rita Izuchukwu.<br />\r\n<br />\r\nManager Lenient Consultant Agency.<br />\r\n<br />\r\n<br />\r\nEmail: rita_izus@lovecat.com<br />\r\nEmail:rita_salicheconsultant@consultant.com<br />\r\n<br />\r\n<br />\r\n'

In [8]:
def lan_detector(s):
        try:
            s.encode(encoding='utf-8').decode('ascii')
            return 'Eng'
        except UnicodeDecodeError:
            return 'Chi'

In [9]:
df['lan']=df['MESSAGE'].apply(lan_detector)

In [210]:
dfc=df[df['lan']=='Chi']
dfe=df[df['lan']=='Eng']

In [211]:
dfe=dfe.reset_index(drop=True)
dfc=dfc.reset_index(drop=True)

In [20]:
dfe.columns

Index(['ENQUIRY_REQUEST_ID', 'MESSAGE_CENTER_ID', 'MESSAGE_TYPE',
       'ENQUIRY_ITEM_TYPE', 'ENQUIRY_TYPE', 'FAIRSYMBOL', 'FAIRFISCALYEAR',
       'EMS_ORDER_NO', 'COMPANY_ID', 'PRODUCT_ID', 'WTMCID', 'DEVICETYPE',
       'ADSYSID', 'CHANNEL (ER)', 'LAST_CHANNEL (AKAMAI)',
       'LAST_CHANNEL_LABEL (AKAMAI)', 'LAST_PUB_CODE (AKAMAI)',
       'LAST_ISSUE_NUM (AKAMAI)', 'SENDER_EMAIL_ADDR', 'SEND_DATE', 'SPAM_IND',
       'STATUS', 'ENQUIRY_NATURE', 'ENQUIRY_NATURE_DESC', 'MESSAGE', 'SUBJECT',
       'ENQUIRY_COUNT_IN_SAME_REQUEST', 'lan'],
      dtype='object')

In [30]:
dfc.groupby('SPAM_IND').count().iloc[:,0]

SPAM_IND
N     35
Y    235
Name: ENQUIRY_REQUEST_ID, dtype: int64

In [32]:
dfe.groupby('SPAM_IND').count().iloc[:,0]

SPAM_IND
N      29
Y    1543
Name: ENQUIRY_REQUEST_ID, dtype: int64

In [45]:
eg1=dfc['MESSAGE'].iloc[0]
eg2=dfc['MESSAGE'].iloc[1]

In [139]:
eg1=dfe['MESSAGE'].iloc[0]
eg4=dfe['MESSAGE'].iloc[3]

In [192]:
def parse_eng(text):
    #initial parsed text
    text=text.lower().replace("\n&nbsp"," ").replace("<br />"," ").replace("\n"," ").replace("\r","").replace("<br>"," ")
    text=re.sub(r'(\w|[@])*.com',"EMAIL",text)
    text=re.sub(r'www.*com',"WEBSITE",text)
    text=re.sub(r'[^\w\s]','',text)
    return text

In [212]:
dfe['p_message']=dfe['MESSAGE'].apply(parse_eng)

In [214]:
dfe['spam']=dfe['SPAM_IND'].apply(lambda x: 1 if x=='Y' else 0)

In [215]:
dfe=dfe[['p_message','spam']]

In [216]:
dfe['spam'].unique()

array([1, 0], dtype=int64)

In [217]:
dfe['p_message'].iloc[:100]

0     please quote me the latest price of the above ...
1     a buyer with the below contact info has just r...
2     please quote me the latest price of the above ...
3     please quote me the latest price of the above ...
4     please quote me the latest price of the above ...
5                                would like to know moq
6     good day    my name is john paul from russia i...
7     good day my name is rita mackenzie from canada...
8     please quote me the latest price of the above ...
9     hi  can you please make me your kind quotation...
10    please quote me the latest price of the above ...
11    please quote me the latest price of the above ...
12    hello    interested in your products we sincer...
13    please quote me the latest price of the above ...
14    dear sirmadam   nbsp nbsp many greetings to yo...
15    please quote me the latest price of the above ...
16    hi  we are EMAILpany based in izmir turkey our...
17    dear supplier    i would like to enquiry a

In [218]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [219]:
vectorizer = TfidfVectorizer("english")
message_mat = vectorizer.fit_transform(list(dfe['p_message']))
message_mat

<1572x4716 sparse matrix of type '<class 'numpy.float64'>'
	with 66018 stored elements in Compressed Sparse Row format>

In [220]:
message_train, message_test, spam_nospam_train, spam_nospam_test = train_test_split(message_mat, dfe['spam'], test_size=0.3, random_state=20)

In [221]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

Spam_model = LogisticRegression(solver='liblinear',penalty='l1')
Spam_model.fit(message_train,spam_nospam_train)
pred = Spam_model.predict(message_test)
accuracy_score(spam_nospam_test,pred)

0.9851694915254238